In [16]:
import pandas as pd
import math

In [14]:
# import the data sets (aprox. 90 sec)
madde = pd.read_excel('https://github.com/aiasci/HHBA/blob/main/first_step/madde.xlsx?raw=true', index_col=False)
DATA_raw = pd.read_excel('https://github.com/aiasci/HHBA/blob/main/first_step/DATA_raw.xlsx?raw=true', index_col = False)
TÜKETİM_raw = pd.read_csv('https://github.com/aiasci/HHBA/blob/main/first_step/T%C3%9CKET%C4%B0M_raw.zip?raw=true', compression = 'zip',index_col = False)

In [22]:
## creating subgroup files
liste = [2015,2016,2017,2018,2019]
for k in liste:   
    DATA_raw = DATA_raw[DATA_raw['BIRIMNO'] != 1803131]
    DATA_raw['GELIR_TOPLAM'] = pd.to_numeric(DATA_raw['GELIR_TOPLAM'], downcast="float")

    data = DATA_raw[DATA_raw['Yıl'] == k]
    data.reset_index(drop=True, inplace=True)
    har = TÜKETİM_raw[TÜKETİM_raw['Yıl'] == k]

    # TÜİK Ağırlıklandırması
    ağırlık = []
    for i in range(0, len(data)):
        if data['FERTNO'][i] == 1:
            ağırlık.append(data['FERTNO'][i])
        if data['FERTNO'][i] != 1:
            if data['YAS'][i] < 14:
                ağırlık.append(0.3)
        if data['FERTNO'][i] != 1:
            if data['YAS'][i] >= 14:
                ağırlık.append(0.5)
    # Veriye Ağırlıkların Eklenmesi
    data['ağırlık'] = ağırlık

    data['GELIR_TOPLAM'] = data['GELIR_TOPLAM'].fillna(0)


    # Gelirlerin HAnelere göre Toplulaştırılması
    hanehalkı_gelirler = data[['BIRIMNO', 'GELIR_TOPLAM','ağırlık']].groupby('BIRIMNO').sum() 
    hanehalkı_gelirler = hanehalkı_gelirler.reset_index()
    # Hane Başına Eşdeğer Fert Gelirlerinin Hesaplanması
    hanehalkı_gelirler['Ortalama'] = hanehalkı_gelirler['GELIR_TOPLAM']/hanehalkı_gelirler['ağırlık']
    # Eşdeğer Fert Gelirlerine göre Hanelerin Sıralanması 
    hanehalkı_gelirler.sort_values('Ortalama', inplace=True)

    # İndex'in düzeltilmesi
    hanehalkı_gelirler =hanehalkı_gelirler.reset_index(drop=True)

    # TÜİK Metodolojisi ile %20'lik gelir gruplarının oluşturulması
    hh_1 = hanehalkı_gelirler[:math.trunc(len(hanehalkı_gelirler)/5)].reset_index(drop=True)
    hh_1['Gelir Grubu'] = 1
    hh_2 = hanehalkı_gelirler[math.trunc(len(hanehalkı_gelirler)/5):math.trunc(len(hanehalkı_gelirler)/5)*2].reset_index(drop=True)
    hh_2['Gelir Grubu'] = 2
    hh_3 = hanehalkı_gelirler[math.trunc(len(hanehalkı_gelirler)/5)*2:math.trunc(len(hanehalkı_gelirler)/5)*3].reset_index(drop=True)
    hh_3['Gelir Grubu'] = 3
    hh_4 = hanehalkı_gelirler[math.trunc(len(hanehalkı_gelirler)/5)*3:math.trunc(len(hanehalkı_gelirler)/5)*4].reset_index(drop=True)
    hh_4['Gelir Grubu'] = 4
    hh_5 = hanehalkı_gelirler[math.trunc(len(hanehalkı_gelirler)/5)*4:].reset_index(drop=True)
    hh_5['Gelir Grubu'] = 5
    # Grupların Toplulaştırılması
    gruplar_main = pd.concat([hh_1, hh_2, hh_3, hh_4, hh_5])

    ##Alt gelir gruplarını oluşturan hanelerin listelenmesi
    a= pd.DataFrame()
    for i in range(1,6):
        vars()['data_hh_'+str(i)+'_'+str(k)] = pd.DataFrame()
        for j in range(0,len(vars()['hh_'+str(i)])):
            a= data[data['BIRIMNO'] == vars()['hh_'+str(i)]['BIRIMNO'][j]]
            vars()['data_hh_'+str(i)+'_'+str(k)] = vars()['data_hh_'+str(i)+'_'+str(k)].append(a)
    ## Alt Gelir Gruplarının Kaydedilmesi
    for i in range(1,6):
        vars()['data_hh_'+str(i)+'_'+str(k)].to_excel(f'grup{i}_{k}.xlsx',index=False)

C:\Users\User\AppData\Local\Temp/ipykernel_4180/2553143296.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ağırlık'] = ağırlık
C:\Users\User\AppData\Local\Temp/ipykernel_4180/2553143296.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['GELIR_TOPLAM'] = data['GELIR_TOPLAM'].fillna(0)
